In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Задание 10. Контест. Steam activity prediction

В этом задании вам предложен набор данных, содержащий информацию об активности пользователей в интернет-магазине компьютерных игр Steam.

Признаковое описание состоит из результатов некоторых запросов в Steam Web API (в частности **GET  http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=XXXXXXXXXXXXXXXXX&steamid=XXXXXXXXXXXXXXX&format=json** и некоторых других: **GetPlayerSummaries (v0002)**, **GetFriendList (v0001)**, **GetPlayerAchievements (v0001)**), которые возвращают объекты, содержащие различные свойства игроков и игр.


1. В файле **users.csv** приводятся некоторые данные о пользователе, в том числе дата его регистрации (timecreated).

2. В файле **users_friends.csv** содержатся пары пользователь-пользователь, обозначающие наличие связи "дружба" между этими пользователями.

3. В файле **achievements_stats.csv** содержатся сведения о достижениях, полученных пользователем в игре. Они представлены в виде списка троек (название_достижения, флаг_скрытого_достижения, процент_пользователей_с_достижением).

Более подробно про признаки можно прочитать в [документации (англ.)](https://developer.valvesoftware.com/wiki/Steam_Web_API#GetPlayerAchievements_.28v0001.29) Steam API.

4. Также среди файлов задания есть файл **games_details.csv**, который получен из API базы знаний об играх RAWG.io. [Документация (англ.)](https://api.rawg.io/docs/) содержит подробные сведения о возвращаемых значениях разных методов, из которых в предложенный файл попали только **genres**, **tags**, **release_date** и **rating**.

**ВНИМАНИЕ! Масштаб и абсолютные значения некоторых признаков были изменены!**

В качестве целевой переменной выбран параметр **playtime_forever** из Steam API, который принимает целые неотрицательные значения и обозначает количество времени (в мин), которое тот или иной пользователь провёл, запустив данную игру.

**ВНИМАНИЕ! Масштаб и абсолютные значения целевой переменной были изменены!**

<font color='red'>**ВНИМАНИЕ! Все данные актуальны на 00:00 9 марта 2023 г.** </font>

В качестве метрики качества используется **[rMSLE (root mean squared logarithmic error)](https://www.kaggle.com/code/carlolepelaars/understanding-the-metric-rmsle/notebook)** -- корень из среднеквадратической ошибки в логарифмической шкале (**обратите внимание на параметр squared=False в вызовах mean_squared_log_error**).

Вам необходимо разработать модель машинного обучения для восстановления зависимости **времени, проведенного пользователем в игре,** от других данных о пользователе и игре.

В папке с заданием помимо этого и вышеописанных файлов находятся:
- 2 файла с данными об известных пользователях и играх -- **train.csv** и **test.csv**, которые содержат, соответственно, тренировочную выборку (с известными значениями целевой переменной в поле **playtime_forever**) и признаковую часть тестовой выборки. Поле **playtime_2weeks** содержит время (в минутах), сыгранное пользователем в игру за последние 2 недели.

В качестве ответа на это задание вы должны предоставить **Kaggle-ноутбук** (как создать такой ноутбук, читайте ниже), который:
1. генерирует на выходе **csv-файл** со столбцом предсказанных значений игрового времени для пар пользователь-игра из тестовой выборки и отправляет его в систему Kaggle. Пример такого файла находится в папке с заданием (**sample_submission.csv**);
2. разрешает чтение пользователю [Sergey Serov](https://www.kaggle.com/ssserov/account).

**НЕВЫПОЛНЕНИЕ ЛЮБОГО ИЗ УКАЗАННЫХ ПУНКТОВ ПРИВЕДЁТ К ОЦЕНИВАНИЮ ЗАДАНИЯ В 0 БАЛЛОВ!**

**Как создать и отправить корректный Kaggle-ноутбук:**

1. На странице соревнования перейдите на вкладку **Code** и нажмите **New Notebook**.
![kaggle_notebook_1.PNG](https://drive.google.com/uc?export=view&id=1Ag0K1plTS5gvQ0XIo7HiOpGQiEXBmhp_)
2. **Никакие дополнительные данные для выполнения задания загружать не нужно** (но это не запрещено). Путь, по которому автоматически находятся необходимые файлы с данными, можно посмотреть, выполнив первую ячейку и изучив ее вывод.
![kaggle_notebook_2](https://drive.google.com/uc?export=view&id=14fva9WSMVqQN5jRQP0KLgaZqx1T-BKRl)

3. Дать права на чтение ноутбука пользователю [Sergey Serov](https://www.kaggle.com/ssserov/account). Для этого в верхней панели ноутбука нужно нажать кнопку **Share**, далее выбрать **Add collaborators** и в поиске найти пользователя Sergey Serov (вместо owner будет написано collaborator). Не забудьте сохранить изменения кнопкой **Save**.
![kaggle_notebook_3.PNG](https://drive.google.com/uc?export=view&id=1-i7WNFnAqQRRQj49VDwOUQSv1EsPPmct)
![kaggle_notebook_4.PNG](https://drive.google.com/uc?export=view&id=18FaPqnZuuvdwCxZ-T4xDnxiTthXLjvxK)

После правильного действия Вы увидите:

![kaggle_notebook_5.jpg](https://drive.google.com/uc?export=view&id=1v0siWcOA3lrIlU--Fma1MLZ5bsITe2C4)

4. Для того, чтобы предсказания, полученные kaggle-ноутбуком были корректно учтены системой, он должен сохранять их следующей командой **submission.to_csv("/kaggle/working/submission.csv", index_label="index")**, где **submission** -- ваш датафрейм с предсказаниями (как в примере ниже).

5. Для отправки ноутбука в правой его панели выберите вкладку **Competitions**, нажмите на кнопку **Submit**, по желанию введите название и описание посылки и подтвердите нажатием кнопки **Submit**.

![kaggle_notebook_6.PNG](https://drive.google.com/uc?export=view&id=1o1aPhDXpymwzJyWjfhBW85ewVXh6ge53)

6. Также этот ноутбук доступен в виде публичного кернела во вкладке **[Code](https://www.kaggle.com/competitions/cmc-ml-steam-activity-prediction/code)** соревнования. Его можно открыть, а затем сразу преобразовать в свой Kaggle-кернел, нажав на три точки справа вверху и выбрав опцию "Copy & edit notebook".
![kaggle_notebook_7.png](https://drive.google.com/uc?export=view&id=1XriF2ZJk4fVhOnisawre9RSJbWVj3LPL)

**Далее в этом ноутбуке покажем пример формирования csv-файла с предсказаниями игрового времени для пар пользователь-игра из тестовой выборки.**

Для начала импортируем библиотеки и загрузим данные из файлов **train.csv** и **test.csv**.

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# Используйте эти пути для запуска ноутбука на Kaggle
PATH_TO_KAGGLE_TRAIN = "/kaggle/input/cmc-ml-steam-activity-prediction/train.csv"
PATH_TO_KAGGLE_TEST = "/kaggle/input/cmc-ml-steam-activity-prediction/test.csv"
PATH_TO_KAGGLE_SUBMISSION = "/kaggle/working/submission.csv"

# Если запускаете на своем компьютере, то впишите актуальные пути
# PATH_TO_TRAIN = # Your Path
# PATH_TO_TEST = # Your Path
# PATH_TO_SUBMISSION = # Your Path
check = pd.read_csv("/kaggle/input/cmc-ml-steam-activity-prediction/achievements_stats.csv")
train = pd.read_csv(PATH_TO_KAGGLE_TRAIN)
achiv = pd.read_csv("/kaggle/input/cmc-ml-steam-activity-prediction/achievements_stats.csv")
X_train = train.drop(["index", "playtime_forever"], axis=1)
y_train = train["playtime_forever"]
friend = pd.read_csv("/kaggle/input/cmc-ml-steam-activity-prediction/users_friends.csv")
X_train.info(), len(y_train)

In [ ]:
game_info = pd.read_csv("/kaggle/input/cmc-ml-steam-activity-prediction/games_details.csv")

In [ ]:
user_friend = friend.groupby("user_id").count().drop(columns= ["Unnamed: 0"]).reset_index()
user_friend

In [ ]:
achiv = achiv.drop_duplicates(subset=['user_id', 'game_id'], keep="first")
achiv

In [ ]:
X_train['playtime_2weeks'].plot(kind= 'hist', edgecolor ='black')

In [ ]:
# def take_achiv(x):
#     if isinstance(x, str):
#         i = 0
#         for item in eval(x):
#             if item[1] == True:
#                 i += 1 -item[2]
#         return i
#     else:
#         return 0.0
def take_achiv(x):
    if isinstance(x, str):
        countT = 0
        y =  eval(x)
        for item in y:
            if item[1] == True:
                countT += 1
        return countT/len(y)
    else:
        return 0.0

In [ ]:
X_train = X_train.merge(achiv[['user_id', 'game_id', "achievements"]],  on = ['user_id', 'game_id'], how = 'left')
X_train["achievements"] =  X_train["achievements"].apply(lambda x: take_achiv(x))
X_train = X_train.merge(game_info[['game_id', "rating"]],  on = ['game_id'], how = 'left')
X_train

In [ ]:
testsss = X_train.merge(user_friend[['user_id', 'friend_id']],  on = ['user_id'], how = 'left')
# X_train = testsss.drop(columns=["achievements_x", "achievements_y"])

X_train = testsss.fillna(testsss.mean())
X_train["rating"][X_train["rating"]==0.0] = X_train["rating"].mean()
X_train["achievements"][X_train["achievements"]==0.0] = X_train["achievements"].mean()
X_train

Затем создадим модель и обучим ее на тренировочной выборке.

In [ ]:

X_train['game_name'] = X_train['game_name'].astype('category')
X_train['user_id'] = X_train['user_id'].astype('category')
X_train['game_id'] = X_train['game_id'].astype('category')
X_train['achievements'] = X_train['achievements'].astype('float')
X_train['friend_id'] = X_train['friend_id'].astype('float')
X_train['rating'] = X_train['rating'].astype('float')
X_train['playtime_2weeks'] = X_train['playtime_2weeks'].astype('float')

In [ ]:
X_train = X_train.drop(["game_name"], axis=1)

In [ ]:
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, y_train,train_size=0.9, random_state=42)

In [ ]:
# https://habr.com/ru/sandbox/163469/
import math
class RMSLE(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        result = []
        for index in range(len(targets)):
            val = max(approxes[index], 0)
            der1 = math.log1p(targets[index]) - math.log1p(max(0, approxes[index]))
            der2 = -1 / (max(0, approxes[index]) + 1)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result
class RMSLE_val(object):
    def get_final_error(self, error, weight):
        return np.sqrt(error / (weight + 1e-38))

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += w * ((math.log1p(max(0, approx[i])) - math.log1p(max(0, target[i])))**2)

        return error_sum, weight_sum

In [ ]:
model = CatBoostRegressor(**{"n_estimators": 175000, "max_depth": 7, "learning_rate":0.001}, 
                          verbose=500, cat_features = ['game_id','user_id'], 
                          l2_leaf_reg = 3.0, random_state=42)
#                           loss_function=RMSLE(),eval_metric=RMSLE_val())
model.fit(X_train, np.log(y_train + 1), eval_set=[(X_test_val.copy(), np.log(y_test_val.copy()+1))])
# model.fit(X_train_val, y_train_val , eval_set=[(X_test_val.copy(), y_test_val.copy())])

Вычислим ошибку модели на тренировочной выборке.

In [ ]:
# pred2 = model.predict(X_test_val)
# pred2 = np.where(pred2>0,pred2, 0)
# print(f"val rMSLE: {mean_squared_log_error(y_test_val, pred2, squared=False)}")

In [ ]:
pred1 = model.predict(X_test_val)
pred1 = np.exp(pred1) - 1
pred1 = np.where(pred1>0,pred1, 0)
print(f"val rMSLE: {mean_squared_log_error(y_test_val, pred1, squared=False)}")

В заключение получим столбец предсказаний игрового времени для тестовой выборки и сохраним его в виде csv-файла (обратите внимание, что в выходном файле должно быть два столбца -- **index** и **playtime_forever**).

In [ ]:
test = pd.read_csv(PATH_TO_KAGGLE_TEST)
test.info()

In [ ]:
test = test.merge(achiv[['user_id', 'game_id', "achievements"]],  on = ['user_id', 'game_id'], how = 'left')
test = test.merge(user_friend[['user_id', 'friend_id']],  on = ['user_id'], how = 'left')
test = test.merge(game_info[['game_id', "rating"]],  on = ['game_id'], how = 'left')
test["achievements"] =  test["achievements"].apply(lambda x: take_achiv(x))
test['game_name'] = test['game_name'].astype('category')
test['user_id'] = test['user_id'].astype('category')
test['game_id'] = test['game_id'].astype('category')
test['friend_id'] = test['friend_id'].astype('float')
test['rating'] = test['rating'].astype('float')
test['achievements'] = test['achievements'].astype('float')
test['playtime_2weeks'] = test['playtime_2weeks'].astype('float')
test = test.fillna(test.mean())
test_index = test["index"]
test = test.drop(["index", "game_name"], axis=1)
test.head()

In [ ]:
pred = model.predict(test)
pred = np.exp(pred) - 1
pred=np.where(pred>0,pred, 0)
submission = pd.DataFrame({"index": test_index, 
                           "playtime_forever": pred})

In [ ]:
submission.to_csv(PATH_TO_KAGGLE_SUBMISSION, index=False)

In [ ]:
import numpy as np
from scipy.stats import sem
from sklearn.metrics import roc_auc_score

y_pred = pred1
y_true = y_test_val.copy().reset_index()["playtime_forever"]
print("Original mean_squared_log_error: {:0.3f}".format(mean_squared_log_error(y_true, y_pred, squared=False)))

n_bootstraps = 1000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred), len(y_pred))
    if len(np.unique(y_true[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue
    score = mean_squared_log_error(y_true[indices], y_pred[indices], squared=False)
    bootstrapped_scores.append(score)
#     print("Bootstrap #{} mean_squared_log_error area: {:0.3f}".format(i + 1, score))


In [ ]:
import matplotlib.pyplot as plt
plt.hist(bootstrapped_scores, bins=50)
plt.title('Histogram of the bootstrapped mean_squared_log_error scores')
plt.show()

In [ ]:
# https://www.kaggle.com/code/tomokikmogura/catboost-hyperparameters-tuning-with-optuna

In [ ]:
import optuna
def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.01, 0.1, 0.01)
    param['depth'] = trial.suggest_int('depth', 6, 9, 1)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0,3.0, 1.0)
#     param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['iterations'] = 5000
    param['use_best_model'] = True
    param['random_state'] = 42
    
    regressor = CatBoostRegressor(**param, cat_features= ["game_name","game_id","user_id"], verbose=False)

    regressor.fit(X_train_val.copy(), np.log(y_train_val + 1),
                  eval_set=[(X_test_val.copy(), np.log(y_test_val + 1))])
    pred = regressor.predict(X_test_val.copy())
    pred=np.exp(pred)-1
    loss = mean_squared_log_error(np.log(y_test_val + 1), np.where(pred>0,pred, 0))
    return loss

In [ ]:
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

In [ ]:
# %%time
# study = optuna.create_study(study_name=f'catboost-seed{42}')
# study.optimize(objective, n_trials=90, n_jobs=-1, timeout=24000)

In [ ]:
# study.best_params

In [ ]:
# {'learning_rate': 0.04, 'depth': 7, 'l2_leaf_reg': 2.0}

In [ ]:
# CatBoostRegressormodel = CatBoostRegressor(**{"iterations": 1000, "max_depth": 7, "learning_rate":0.02, 'l2_leaf_reg': 1.0}, 
#                           verbose=False, cat_features = ['game_name','game_id','user_id'], random_state=42,
#                           loss_function=RMSLE(),eval_metric=RMSLE_val())
# CatBoostRegressormodel.fit(X_train_val, y_train_val)

In [ ]:

# pred = CatBoostRegressormodel.predict(X_test_val)
# pred = np.where(pred >= 0, pred, 0)
# print(f"Train rMSLE: {mean_squared_log_error(y_test_val, pred, squared=False)}")

In [ ]:
# test = pd.read_csv(PATH_TO_KAGGLE_TEST)
# test["achievements"] =  test["achievements"].apply(lambda x: take_achiv(x))
# test['game_name'] = test['game_name'].astype('category')
# test['user_id'] = test['user_id'].astype('category')
# test['game_id'] = test['game_id'].astype('category')
# test['achievements'] = test['achievements'].astype('float')
# test['playtime_2weeks'] = test['playtime_2weeks'].astype('float')
# test_index = test["index"]
# test = test.drop(["index"], axis=1)
# test.info()
# test.head()

In [ ]:
# pred = CatBoostRegressormodel.predict(test)
# pred = np.where(pred >= 0, pred, 0)
# submission = pd.DataFrame({"index": test_index, 
#                            "playtime_forever": pred})

In [ ]:
# submission.to_csv(PATH_TO_KAGGLE_SUBMISSION, index=False)